# MQ-5

## Specifications

### Standard Work Condition

    Vc (Circuit Voltage)       :  5v +/- 0.1
    Vh (Heating Voltage)       :  5v +/- 0.1
    Rl (Load Resistance)       :  20KOhm
    Rh (Heater Resistance)     :  33Ohm +/- 5%
    Ph (Heating consumption)   :  less than 800mw
    

### Sensitivity Characteristic:

    Rs (Sensing Resistance)    : 10kOhm  -  60kOhm (5000ppm methane)
    Preheat Time               : Over 24 hour
    
    Detecting Concentration Scope:
        LPG, LNG, Natuarl Gas, Propane and Town Gas:  200ppm  - 10000ppm 


In [1]:
#%pip install numpy

import numpy as np
import math

In [2]:
## Gas concentration values referenced in the datasheet

# Hydrogen
h2 = {
    'p1' : {'x': 200,   'y':1.730},
    'p2' : {'x': 500,   'y':1.215},
    'p3' : {'x': 800,   'y':1.050},
    'p4' : {'x': 1000,  'y':0.988},
    'p5' : {'x': 1500,  'y':0.875},
    'p6' : {'x': 2000,  'y':0.823},
    'p7' : {'x': 3000,  'y':0.765},
    'p8' : {'x': 5000,  'y':0.728},
    'p9' : {'x': 10000, 'y':0.657}
}

# Liquefied Petroleum Gas
lpg ={
    'p1' : {'x': 200,   'y':0.698},
    'p2' : {'x': 500,   'y':0.473},
    'p3' : {'x': 800,   'y':0.389},
    'p4' : {'x': 1000,  'y':0.355},
    'p5' : {'x': 1500,  'y':0.297},
    'p6' : {'x': 2000,  'y':0.268},
    'p7' : {'x': 3000,  'y':0.230},
    'p8' : {'x': 5000,  'y':0.189},
    'p9' : {'x': 10000, 'y':0.145}
}

# Methane
ch4 ={
    'p1' : {'x': 200,   'y':0.952},
    'p2' : {'x': 500,   'y':0.673},
    'p3' : {'x': 800,   'y':0.554},
    'p4' : {'x': 1000,  'y':0.515},
    'p5' : {'x': 1500,  'y':0.445},
    'p6' : {'x': 2000,  'y':0.400},
    'p7' : {'x': 3000,  'y':0.340},
    'p8' : {'x': 5000,  'y':0.275},
    'p9' : {'x': 10000, 'y':0.204}
}

# Carbon Monoxide
co = {
    'p1' : {'x': 200,   'y':3.890},
    'p2' : {'x': 500,   'y':3.201},
    'p3' : {'x': 800,   'y':3.012},
    'p4' : {'x': 1000,  'y':2.886},
    'p5' : {'x': 1500,  'y':2.666},
    'p6' : {'x': 2000,  'y':2.555},
    'p7' : {'x': 3000,  'y':2.448},
    'p8' : {'x': 5000,  'y':2.375},
    'p9' : {'x': 10000, 'y':2.276}
}

alcohol = {
    'p1' : {'x': 200,   'y':3.507},
    'p2' : {'x': 500,   'y':2.921},
    'p3' : {'x': 800,   'y':2.666},
    'p4' : {'x': 1000,  'y':2.555},
    'p5' : {'x': 1500,  'y':2.332},
    'p6' : {'x': 2000,  'y':2.235},
    'p7' : {'x': 3000,  'y':2.039},
    'p8' : {'x': 5000,  'y':1.817},
    'p9' : {'x': 10000, 'y':1.441}
}

AIR = 6.464

In [3]:
# RS value calculation
Rs    = lambda Vcc, Vrl, RL : ((Vcc * RL) / Vrl) - RL

# R0 value calculation
R0    = lambda Rs, air : Rs / air

# RS/Ro value calculation (ratio)
Ratio = lambda Rs, R0: Rs / R0

# log(y) = m.log(x) + b
# m
M     = lambda p1, p2 : (math.log10( p2['y'] / p1['y'] )) / (math.log10(p2['x']/p1['x']))
# b
B     = lambda p, m : math.log10(p['y']) - (m * math.log10(p['x']))

# PPM value calculation 
PPM   = lambda  racio, m, b: 10 ** ((math.log10(racio) - b) / m)

In [4]:
## Calculate the points to be used for the calculation of _m_ and _b_ according to the _ratio_ value 
def getPoints(racio, values):
    size = len(values)
    for idx in range(size - 1):
        if racio >= values[idx]['y']:
            if idx == 0:
                return (values[0], values[1])
            return(values[idx - 1], values[idx])
    return(values[size - 2], values[size - 1])

In [5]:
## Test the _getPoints_ function
"""
points = ['p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9']
_h2  = [ h2[n]      for n in list(h2)      if n in points]


racio  = 1
p1, p2  = getPoints(racio, _h2)

m = M(p1,p2)
b = B(p1, m)

h2_ppm  = PPM(racio, m, b)
print(f"{p1}\n{p2}")
print(f"[ m: {m:.2f} || b: {b:.2f} ]")
print(f"{h2_ppm:.0f}ppm")
"""
print()

In [6]:
points = ['p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9']

_h2      = [ h2[n]      for n in list(h2)      if n in points]
_lpg     = [ lpg[n]     for n in list(lpg)     if n in points]
_ch4     = [ ch4[n]     for n in list(ch4)     if n in points]
_co      = [ co[n]      for n in list(co)      if n in points]
_alcohol = [ alcohol[n] for n in list(alcohol) if n in points]

# print(_h2)
# print(_lpg)
# print(_ch4)
# print(_co)
# print(_alcohol)


In [7]:
_ratios   = [10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]

h2_ppm      = []
lpg_ppm     = []
ch4_ppm     = []
co_ppm      = []
alcohol_ppm = []

for r in _ratios:
    p1, p2  = getPoints(r, _h2)
    m = M(p1,p2)
    b = B(p1, m)
    h2_ppm.append(PPM(r, m, b))
    
    p1, p2  = getPoints(r, _lpg)
    m = M(p1,p2)
    b = B(p1, m)
    lpg_ppm.append(PPM(r, m, b))
    
    p1, p2  = getPoints(r, _ch4)
    m = M(p1,p2)
    b = B(p1, m)
    ch4_ppm.append(PPM(r, m, b))
    
    p1, p2  = getPoints(r, _co)
    m = M(p1,p2)
    b = B(p1, m)
    co_ppm.append(PPM(r, m, b))
    
    p1, p2  = getPoints(r, _alcohol)
    m = M(p1,p2)
    b = B(p1, m)
    alcohol_ppm.append(PPM(r, m, b))
    


#print(f"{[f'{str(i).center(6)}' for i in _ratio]}")
    
print ("{:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format('Ratio','H2', 'LPG', 'CH4', 'CO', 'Alcohol'))
for i in range(len(_ratios)):
    if _ratios[i] > 1:
        print ("{:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
               .format( _ratios[i],
                       round(h2_ppm[i],3),
                       round(lpg_ppm[i],3),
                       round(ch4_ppm[i],3),
                       round(co_ppm[i],3),
                       round(alcohol_ppm[i],3)
                      ))
    else:
        print ("{:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
               .format( _ratios[i],
                       round(h2_ppm[i],3),
                       round(lpg_ppm[i],3),
                       round(ch4_ppm[i],3),
                       "----",
                       round(alcohol_ppm[i],3)
                      ))

Ratio    H2            LPG           CH4           CO            Alcohol      
10       2.115         0.379         0.401         2.364         1.048        
9        2.779         0.486         0.529         3.879         1.777        
8        3.772         0.641         0.722         6.748         3.207        
7        5.333         0.878         1.028         12.641        6.262        
6        7.954         1.262         1.544         26.088        13.56        
5        12.761        1.938         2.5           61.462        33.812       
4        22.759        3.278         4.508         175.432       103.454      
3        47.987        6.454         9.639         816.852       437.409      
2        137.314       16.769        28.137        82029.08      3268.051     
1        956.703       85.772        175.626       ----          29809.087    
0.9      1365.366      109.925       231.996       ----          40845.785    
0.8      2340.602      145.06        316.682       -

In [8]:
# MQ 5 - sensor 1
sensor1 = {}

sensor1['Vcc'] = 5      #volts
sensor1['Vrl'] = 0.26   #volts
sensor1['RL']  = 10     #kOhms
sensor1['AIR'] = AIR   

sensor1['rs'] = Rs(sensor1['Vcc'], sensor1['Vrl'], sensor1['RL'])
sensor1['ro'] = R0(sensor1['rs'], sensor1['AIR'])
sensor1['ratio'] = Ratio(sensor1['rs'], sensor1['ro'])

# MQ 5 - sensor 2
sensor2 = {}

sensor2['Vcc'] = 5      #volts
sensor2['Vrl'] = 0.38   #volts
sensor2['RL']  = 10     #kOhms
sensor2['AIR'] = AIR   

sensor2['rs'] = Rs(sensor2['Vcc'], sensor2['Vrl'], sensor2['RL'])
sensor2['ro'] = R0(sensor2['rs'], sensor2['AIR'])
sensor2['ratio'] = Ratio(sensor2['rs'], sensor2['ro'])



print ("{:<8} {:<8} {:<8} {:<8}".format('Sensor','Rs', 'Ro','Ratio'))
print ("{:<8} {:<8} {:<8} {:<8}".format(1, round(sensor1['rs'],2), round(sensor1['ro'], 2), round(sensor1['ratio'], 2)))
print ("{:<8} {:<8} {:<8} {:<8}".format(2, round(sensor2['rs'],2), round(sensor2['ro'], 2), round(sensor2['ratio'], 2)))
    

Sensor   Rs       Ro       Ratio   
1        182.31   28.2     6.46    
2        121.58   18.81    6.46    


In [9]:
## PPM
sensor1['h2'] = {}
sensor1['h2']['p1'], sensor1['h2']['p2'] = getPoints(sensor1['ratio'], _h2)
sensor1['h2']['m'] = M(sensor1['h2']['p1'], sensor1['h2']['p2'])
sensor1['h2']['b'] = B(sensor1['h2']['p1'], sensor1['h2']['m'])


sensor1['lpg'] = {}
sensor1['lpg']['p1'], sensor1['lpg']['p2'] = getPoints(sensor1['ratio'], _lpg)
sensor1['lpg']['m'] = M(sensor1['lpg']['p1'], sensor1['lpg']['p2'])
sensor1['lpg']['b'] = B(sensor1['lpg']['p1'], sensor1['lpg']['m'])


sensor1['ch4'] = {}
sensor1['ch4']['p1'], sensor1['ch4']['p2'] = getPoints(sensor1['ratio'], _ch4)
sensor1['ch4']['m'] = M(sensor1['ch4']['p1'], sensor1['ch4']['p2'])
sensor1['ch4']['b'] = B(sensor1['ch4']['p1'], sensor1['ch4']['m'])


sensor1['co'] = {}
sensor1['co']['p1'], sensor1['co']['p2'] = getPoints(sensor1['ratio'], _co)
sensor1['co']['m'] = M(sensor1['co']['p1'], sensor1['co']['p2'])
sensor1['co']['b'] = B(sensor1['co']['p1'], sensor1['co']['m'])


sensor1['alcohol'] = {}
sensor1['alcohol']['p1'], sensor1['alcohol']['p2'] = getPoints(sensor1['ratio'], _alcohol)
sensor1['alcohol']['m'] = M(sensor1['alcohol']['p1'], sensor1['alcohol']['p2'])
sensor1['alcohol']['b'] = B(sensor1['alcohol']['p1'], sensor1['alcohol']['m'])


####

sensor2['h2'] = {}
sensor2['h2']['p1'], sensor2['h2']['p2'] = getPoints(sensor2['ratio'], _h2)
sensor2['h2']['m'] = M(sensor2['h2']['p1'], sensor2['h2']['p2'])
sensor2['h2']['b'] = B(sensor2['h2']['p1'], sensor2['h2']['m'])


sensor2['lpg'] = {}
sensor2['lpg']['p1'], sensor2['lpg']['p2'] = getPoints(sensor2['ratio'], _lpg)
sensor2['lpg']['m'] = M(sensor2['lpg']['p1'], sensor2['lpg']['p2'])
sensor2['lpg']['b'] = B(sensor2['lpg']['p1'], sensor2['lpg']['m'])


sensor2['ch4'] = {}
sensor2['ch4']['p1'], sensor2['ch4']['p2'] = getPoints(sensor2['ratio'], _ch4)
sensor2['ch4']['m'] = M(sensor2['ch4']['p1'], sensor2['ch4']['p2'])
sensor2['ch4']['b'] = B(sensor2['ch4']['p1'], sensor2['ch4']['m'])


sensor2['co'] = {}
sensor2['co']['p1'], sensor2['co']['p2'] = getPoints(sensor2['ratio'], _co)
sensor2['co']['m'] = M(sensor2['co']['p1'], sensor2['co']['p2'])
sensor2['co']['b'] = B(sensor2['co']['p1'], sensor2['co']['m'])


sensor2['alcohol'] = {}
sensor2['alcohol']['p1'], sensor2['alcohol']['p2'] = getPoints(sensor2['ratio'], _alcohol)
sensor2['alcohol']['m'] = M(sensor2['alcohol']['p1'], sensor2['alcohol']['p2'])
sensor2['alcohol']['b'] = B(sensor2['alcohol']['p1'], sensor2['alcohol']['m'])


####

print ("{:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format('Volt', 'Ratio','H2', 'LPG', 'CH4', 'CO', 'Alcohol'))

print ("{:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
   .format(round(sensor1['Vrl'], 3),
           round(sensor1['ratio'], 2),
           round(PPM(sensor1['ratio'], sensor1['h2']['m'],      sensor1['h2']['b']),2),
           round(PPM(sensor1['ratio'], sensor1['lpg']['m'],     sensor1['lpg']['b']),2),
           round(PPM(sensor1['ratio'], sensor1['ch4']['m'],     sensor1['ch4']['b']),2),
           round(PPM(sensor1['ratio'], sensor1['co']['m'],      sensor1['co']['b']),2),
           round(PPM(sensor1['ratio'], sensor1['alcohol']['m'], sensor1['alcohol']['b']),2),
          ))

print ("{:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
   .format(round(sensor2['Vrl'], 3),
           round(sensor2['ratio'], 2),
           round(PPM(sensor2['ratio'], sensor2['h2']['m'],      sensor2['h2']['b']),2),
           round(PPM(sensor2['ratio'], sensor2['lpg']['m'],     sensor2['lpg']['b']),2),
           round(PPM(sensor2['ratio'], sensor2['ch4']['m'],     sensor2['ch4']['b']),2),
           round(PPM(sensor2['ratio'], sensor2['co']['m'],      sensor2['co']['b']),2),
           round(PPM(sensor2['ratio'], sensor2['alcohol']['m'], sensor2['alcohol']['b']),2),
          ))

Volt     Ratio    H2            LPG           CH4           CO            Alcohol      
0.26     6.46     6.56          1.06          1.27          18.38         9.34         
0.38     6.46     6.56          1.06          1.27          18.38         9.34         


In [10]:
## Projection - Increasing VRL

# Sensor 1

Vcc    = 5 #volts
vrl    = np.arange(sensor1['Vrl'], 4.5, 0.1)
RL     = 10 #kOhms 

projection = {}
projection['rs'] = [Rs(Vcc, v, RL) for v in vrl]
projection['ro'] = sensor1['ro']
projection['ratio'] = [Ratio(rs, projection['ro']) for rs in projection['rs']]

#print(projection['rs'])
#print(projection['ratio'])

print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format('', 'Volt', 'Ratio','H2', 'LPG', 'CH4', 'CO', 'Alcohol'))

for idx in range(len(projection['ratio'])):

    r = projection['ratio'][idx]
    projection['h2'] = {}
    projection['h2']['p1'], projection['h2']['p2'] = getPoints(r, _h2)
    projection['h2']['m'] = M(projection['h2']['p1'], projection['h2']['p2'])
    projection['h2']['b'] = B(projection['h2']['p1'], projection['h2']['m'])
    
    projection['lpg'] = {}
    projection['lpg']['p1'], projection['lpg']['p2'] = getPoints(r, _lpg)
    projection['lpg']['m'] = M(projection['lpg']['p1'], projection['lpg']['p2'])
    projection['lpg']['b'] = B(projection['lpg']['p1'], projection['lpg']['m'])


    projection['ch4'] = {}
    projection['ch4']['p1'], projection['ch4']['p2'] = getPoints(r, _ch4)
    projection['ch4']['m'] = M(projection['ch4']['p1'], projection['ch4']['p2'])
    projection['ch4']['b'] = B(projection['ch4']['p1'], projection['ch4']['m'])


    projection['co'] = {}
    projection['co']['p1'], projection['co']['p2'] = getPoints(r, _co)
    projection['co']['m'] = M(projection['co']['p1'], projection['co']['p2'])
    projection['co']['b'] = B(projection['co']['p1'], projection['co']['m'])


    projection['alcohol'] = {}
    projection['alcohol']['p1'], projection['alcohol']['p2'] = getPoints(r, _alcohol)
    projection['alcohol']['m'] = M(projection['alcohol']['p1'], projection['alcohol']['p2'])
    projection['alcohol']['b'] = B(projection['alcohol']['p1'], projection['alcohol']['m'])

    if r > 1.20:
        print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
           .format(idx,
                   round(vrl[idx], 3),
                   round(r, 2),
                   round(PPM(r, projection['h2']['m'], projection['h2']['b']),2),
                   round(PPM(r, projection['lpg']['m'], projection['lpg']['b']),2),
                   round(PPM(r, projection['ch4']['m'], projection['ch4']['b']),2),
                   round(PPM(r, projection['co']['m'], projection['co']['b']),2),
                   round(PPM(r, projection['alcohol']['m'], projection['alcohol']['b']),2),
                  ))
    elif r > 0.1:
        print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
           .format(idx,
                   round(vrl[idx], 3),
                   round(r, 2),
                   round(PPM(r, projection['h2']['m'], projection['h2']['b']),2),
                   round(PPM(r, projection['lpg']['m'], projection['lpg']['b']),2),
                   round(PPM(r, projection['ch4']['m'], projection['ch4']['b']),2),
                   "----",
                   round(PPM(r, projection['alcohol']['m'], projection['alcohol']['b']),2),
                  ))
    else:
        print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
           .format(idx,
                   round(vrl[idx], 3),
                   round(r, 2),
                   "----",
                   round(PPM(r, projection['lpg']['m'], projection['lpg']['b']),2),
                   round(PPM(r, projection['ch4']['m'], projection['ch4']['b']),2),
                   "----",
                   round(PPM(r, projection['alcohol']['m'], projection['alcohol']['b']),2),
                  ))


    

    Volt     Ratio    H2            LPG           CH4           CO            Alcohol      
0   0.26     6.46     6.56          1.06          1.27          18.38         9.34         
1   0.36     4.57     16.11         2.4           3.17          93.8          53.07        
2   0.46     3.5      32.19         4.49          6.42          328.87        202.18       
3   0.56     2.81     56.8          7.52          11.45         1143.72       608.96       
4   0.66     2.33     92.26         11.69         18.76         6753.91       1502.02      
5   0.76     1.98     141.29        17.21         28.97         98131.01      3431.45      
6   0.86     1.71     207.11        24.35         42.77         1082715.11    6027.8       
7   0.96     1.49     293.49        33.43         61.01         9662793.53    9010.09      
8   1.06     1.32     404.96        44.78         84.69         72927205.76   13059.61     
9   1.16     1.17     558.85        58.82         115.02        ----          18

In [11]:
## Projection - Increasing VRL

# Sensor 2

Vcc    = 5 #volts
vrl    = np.arange(sensor2['Vrl'], 4.5, 0.1)
RL     = 10 #kOhms 

projection = {}
projection['rs'] = [Rs(Vcc, v, RL) for v in vrl]
projection['ro'] = sensor2['ro']
projection['ratio'] = [Ratio(rs, projection['ro']) for rs in projection['rs']]

#print(projection['rs'])
#print(projection['ratio'])

print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format('', 'Volt', 'Ratio','H2', 'LPG', 'CH4', 'CO', 'Alcohol'))

for idx in range(len(projection['ratio'])):

    r = projection['ratio'][idx]
    projection['h2'] = {}
    projection['h2']['p1'], projection['h2']['p2'] = getPoints(r, _h2)
    projection['h2']['m'] = M(projection['h2']['p1'], projection['h2']['p2'])
    projection['h2']['b'] = B(projection['h2']['p1'], projection['h2']['m'])
    
    projection['lpg'] = {}
    projection['lpg']['p1'], projection['lpg']['p2'] = getPoints(r, _lpg)
    projection['lpg']['m'] = M(projection['lpg']['p1'], projection['lpg']['p2'])
    projection['lpg']['b'] = B(projection['lpg']['p1'], projection['lpg']['m'])


    projection['ch4'] = {}
    projection['ch4']['p1'], projection['ch4']['p2'] = getPoints(r, _ch4)
    projection['ch4']['m'] = M(projection['ch4']['p1'], projection['ch4']['p2'])
    projection['ch4']['b'] = B(projection['ch4']['p1'], projection['ch4']['m'])


    projection['co'] = {}
    projection['co']['p1'], projection['co']['p2'] = getPoints(r, _co)
    projection['co']['m'] = M(projection['co']['p1'], projection['co']['p2'])
    projection['co']['b'] = B(projection['co']['p1'], projection['co']['m'])


    projection['alcohol'] = {}
    projection['alcohol']['p1'], projection['alcohol']['p2'] = getPoints(r, _alcohol)
    projection['alcohol']['m'] = M(projection['alcohol']['p1'], projection['alcohol']['p2'])
    projection['alcohol']['b'] = B(projection['alcohol']['p1'], projection['alcohol']['m'])

    if r > 1.20:
        print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
           .format(idx,
                   round(vrl[idx], 3),
                   round(r, 2),
                   round(PPM(r, projection['h2']['m'], projection['h2']['b']),2),
                   round(PPM(r, projection['lpg']['m'], projection['lpg']['b']),2),
                   round(PPM(r, projection['ch4']['m'], projection['ch4']['b']),2),
                   round(PPM(r, projection['co']['m'], projection['co']['b']),2),
                   round(PPM(r, projection['alcohol']['m'], projection['alcohol']['b']),2),
                  ))
    elif r > 0.1:
        print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
           .format(idx,
                   round(vrl[idx], 3),
                   round(r, 2),
                   round(PPM(r, projection['h2']['m'], projection['h2']['b']),2),
                   round(PPM(r, projection['lpg']['m'], projection['lpg']['b']),2),
                   round(PPM(r, projection['ch4']['m'], projection['ch4']['b']),2),
                   "----",
                   round(PPM(r, projection['alcohol']['m'], projection['alcohol']['b']),2),
                  ))
    else:
        print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
           .format(idx,
                   round(vrl[idx], 3),
                   round(r, 2),
                   "----",
                   round(PPM(r, projection['lpg']['m'], projection['lpg']['b']),2),
                   round(PPM(r, projection['ch4']['m'], projection['ch4']['b']),2),
                   "----",
                   round(PPM(r, projection['alcohol']['m'], projection['alcohol']['b']),2),
                  ))


    

    Volt     Ratio    H2            LPG           CH4           CO            Alcohol      
0   0.38     6.46     6.56          1.06          1.27          18.38         9.34         
1   0.48     5.01     12.72         1.93          2.49          61.08         33.59        
2   0.58     4.05     22.01         3.18          4.36          165.16        97.01        
3   0.68     3.38     35.29         4.88          7.05          388.42        241.44       
4   0.78     2.88     53.51         7.13          10.77         1017.05       541.12       
5   0.88     2.49     77.86         10.02         15.78         2561.21       1122.84      
6   0.98     2.18     109.7         13.67         22.38         20028.48      2228.49      
7   1.08     1.93     150.65        18.24         30.92         146792.01     3829.0       
8   1.18     1.72     202.67        23.88         41.84         945112.33     5879.2       
9   1.28     1.55     268.08        30.79         55.63         5472069.72    81

In [12]:
## ProjectionRo Ro - Fixing VRL and Varying the Ro

# Sensor 1

Vcc    = 5 #volts
Vrl    = sensor1['Vrl']
RL     = 10 #kOhms 

projectionRo = {}
projectionRo['rs'] = sensor1['rs']   
projectionRo['ro'] = np.arange(sensor1['ro'] - 1, sensor1['ro'] + 1, 0.1)
projectionRo['ratio'] = [ Ratio(projectionRo['rs'], ro) for ro in projectionRo['ro']] 

#print(projectionRo['rs'])
#print(projectionRo['ratio'])

print ("{:<3}  {:<8} {:<8} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format('', 'Volt','Rs', 'Ro', 'Ratio','H2', 'LPG', 'CH4', 'CO', 'Alcohol'))

for idx in range(len(projectionRo['ratio'])):

    r = projectionRo['ratio'][idx]
    projectionRo['h2'] = {}
    projectionRo['h2']['p1'], projectionRo['h2']['p2'] = getPoints(r, _h2)
    projectionRo['h2']['m'] = M(projectionRo['h2']['p1'], projectionRo['h2']['p2'])
    projectionRo['h2']['b'] = B(projectionRo['h2']['p1'], projectionRo['h2']['m'])
    
    projectionRo['lpg'] = {}
    projectionRo['lpg']['p1'], projectionRo['lpg']['p2'] = getPoints(r, _lpg)
    projectionRo['lpg']['m'] = M(projectionRo['lpg']['p1'], projectionRo['lpg']['p2'])
    projectionRo['lpg']['b'] = B(projectionRo['lpg']['p1'], projectionRo['lpg']['m'])


    projectionRo['ch4'] = {}
    projectionRo['ch4']['p1'], projectionRo['ch4']['p2'] = getPoints(r, _ch4)
    projectionRo['ch4']['m'] = M(projectionRo['ch4']['p1'], projectionRo['ch4']['p2'])
    projectionRo['ch4']['b'] = B(projectionRo['ch4']['p1'], projectionRo['ch4']['m'])


    projectionRo['co'] = {}
    projectionRo['co']['p1'], projectionRo['co']['p2'] = getPoints(r, _co)
    projectionRo['co']['m'] = M(projectionRo['co']['p1'], projectionRo['co']['p2'])
    projectionRo['co']['b'] = B(projectionRo['co']['p1'], projectionRo['co']['m'])


    projectionRo['alcohol'] = {}
    projectionRo['alcohol']['p1'], projectionRo['alcohol']['p2'] = getPoints(r, _alcohol)
    projectionRo['alcohol']['m'] = M(projectionRo['alcohol']['p1'], projectionRo['alcohol']['p2'])
    projectionRo['alcohol']['b'] = B(projectionRo['alcohol']['p1'], projectionRo['alcohol']['m'])


    print ("{:<3} {:<8} {:<8} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format(idx,
               round(Vrl, 3),
               round(projectionRo['rs'],2),
               round(projectionRo['ro'][idx],2),
               round(r, 2),
               round(PPM(r, projectionRo['h2']['m'], projectionRo['h2']['b']),2),
               round(PPM(r, projectionRo['lpg']['m'], projectionRo['lpg']['b']),2),
               round(PPM(r, projectionRo['ch4']['m'], projectionRo['ch4']['b']),2),
               round(PPM(r, projectionRo['co']['m'], projectionRo['co']['b']),2),
               round(PPM(r, projectionRo['alcohol']['m'], projectionRo['alcohol']['b']),2),
              ))

     Volt     Rs       Ro       Ratio    H2            LPG           CH4           CO            Alcohol      
0   0.26     182.31   27.2     6.7      5.97          0.97          1.15          15.51         7.79         
1   0.26     182.31   27.3     6.68     6.03          0.98          1.16          15.78         7.93         
2   0.26     182.31   27.4     6.65     6.09          0.99          1.18          16.06         8.08         
3   0.26     182.31   27.5     6.63     6.14          1.0           1.19          16.33         8.23         
4   0.26     182.31   27.6     6.6      6.2           1.01          1.2           16.61         8.38         
5   0.26     182.31   27.7     6.58     6.26          1.02          1.21          16.9          8.53         
6   0.26     182.31   27.8     6.56     6.32          1.02          1.22          17.19         8.69         
7   0.26     182.31   27.9     6.53     6.38          1.03          1.23          17.48         8.85         
8   0.26 

In [13]:
## ProjectionRo Ro - Fixing VRL and Varying the Ro

# Sensor 1

Vcc    = 5 #volts
Vrl    = sensor2['Vrl']
RL     = 10 #kOhms 

projectionRo = {}
projectionRo['rs'] = sensor2['rs']   
projectionRo['ro'] = np.arange(sensor2['ro'] - 1, sensor2['ro'] + 1, 0.1)
projectionRo['ratio'] = [ Ratio(projectionRo['rs'], ro) for ro in projectionRo['ro']] 

#print(projectionRo['rs'])
#print(projectionRo['ratio'])

print ("{:<3}  {:<8} {:<8} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format('', 'Volt','Rs', 'Ro', 'Ratio','H2', 'LPG', 'CH4', 'CO', 'Alcohol'))

for idx in range(len(projectionRo['ratio'])):

    r = projectionRo['ratio'][idx]
    projectionRo['h2'] = {}
    projectionRo['h2']['p1'], projectionRo['h2']['p2'] = getPoints(r, _h2)
    projectionRo['h2']['m'] = M(projectionRo['h2']['p1'], projectionRo['h2']['p2'])
    projectionRo['h2']['b'] = B(projectionRo['h2']['p1'], projectionRo['h2']['m'])
    
    projectionRo['lpg'] = {}
    projectionRo['lpg']['p1'], projectionRo['lpg']['p2'] = getPoints(r, _lpg)
    projectionRo['lpg']['m'] = M(projectionRo['lpg']['p1'], projectionRo['lpg']['p2'])
    projectionRo['lpg']['b'] = B(projectionRo['lpg']['p1'], projectionRo['lpg']['m'])


    projectionRo['ch4'] = {}
    projectionRo['ch4']['p1'], projectionRo['ch4']['p2'] = getPoints(r, _ch4)
    projectionRo['ch4']['m'] = M(projectionRo['ch4']['p1'], projectionRo['ch4']['p2'])
    projectionRo['ch4']['b'] = B(projectionRo['ch4']['p1'], projectionRo['ch4']['m'])


    projectionRo['co'] = {}
    projectionRo['co']['p1'], projectionRo['co']['p2'] = getPoints(r, _co)
    projectionRo['co']['m'] = M(projectionRo['co']['p1'], projectionRo['co']['p2'])
    projectionRo['co']['b'] = B(projectionRo['co']['p1'], projectionRo['co']['m'])


    projectionRo['alcohol'] = {}
    projectionRo['alcohol']['p1'], projectionRo['alcohol']['p2'] = getPoints(r, _alcohol)
    projectionRo['alcohol']['m'] = M(projectionRo['alcohol']['p1'], projectionRo['alcohol']['p2'])
    projectionRo['alcohol']['b'] = B(projectionRo['alcohol']['p1'], projectionRo['alcohol']['m'])


    print ("{:<3} {:<8} {:<8} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format(idx,
               round(Vrl, 3),
               round(projectionRo['rs'],2),
               round(projectionRo['ro'][idx],2),
               round(r, 2),
               round(PPM(r, projectionRo['h2']['m'], projectionRo['h2']['b']),2),
               round(PPM(r, projectionRo['lpg']['m'], projectionRo['lpg']['b']),2),
               round(PPM(r, projectionRo['ch4']['m'], projectionRo['ch4']['b']),2),
               round(PPM(r, projectionRo['co']['m'], projectionRo['co']['b']),2),
               round(PPM(r, projectionRo['alcohol']['m'], projectionRo['alcohol']['b']),2),
              ))

     Volt     Rs       Ro       Ratio    H2            LPG           CH4           CO            Alcohol      
0   0.38     121.58   17.81    6.83     5.69          0.93          1.1           14.22         7.1          
1   0.38     121.58   17.91    6.79     5.77          0.94          1.11          14.6          7.3          
2   0.38     121.58   18.01    6.75     5.86          0.96          1.13          14.99         7.51         
3   0.38     121.58   18.11    6.71     5.94          0.97          1.15          15.38         7.72         
4   0.38     121.58   18.21    6.68     6.03          0.98          1.16          15.78         7.94         
5   0.38     121.58   18.31    6.64     6.11          0.99          1.18          16.2          8.16         
6   0.38     121.58   18.41    6.6      6.2           1.01          1.2           16.62         8.38         
7   0.38     121.58   18.51    6.57     6.29          1.02          1.22          17.04         8.61         
8   0.38 